In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu b

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-06-27 00:47:05--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-06-27 00:47:06 (6.54 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [6]:
#IMPORTING TABLES

# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")

customers_df.show()

# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"]).drop_duplicates()

products_df.show()

# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])

review_id_df.show()

# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])

vine_df.show()


+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   10270641|             1|
|   18365872|             1|
|   16711087|             1|
|   10742726|             2|
|   41169638|             1|
|   43622307|             1|
|   24540309|             2|
|   28258386|             1|
|   35329257|             2|
|   14552054|             1|
|   14529507|             5|
|   45392827|             5|
|   47282953|             1|
|    8201930|             1|
|   20109760|             2|
|   16405801|             4|
|   15056685|            21|
|   20840575|             2|
|   39048303|             1|
|    5596610|             1|
+-----------+--------------+
only showing top 20 rows

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00134HSYS|Special Edition P...|
|B00BS6NPBG|High Tech Pet 6-P...|
|B000F930FS|PetSafe ScoopFree...|
|B000FJ9QTW|Insight ActiviToy...|
|B000ALY0OQ|SmartCat Bootsie'...|
|B00Q8

In [7]:
#### DELIVERABLE 2 ###### 
#1 TOTAL VOTES OVER 20 

over20_df = df.filter(df.total_votes >= 20)

over20_df.show()



+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51227803|R21KC552Y6HL8X|B00E2E1XBK|     180836080|Kennel Deck - 3 Pack|    Pet Products|          1|           27|         31|   N|                Y|The picture is of...|The product add p...| 2015-08-31|
|         US|   52747799| RX9WC9FTIR1XR|B00DZITYGU|     598776873|Grey Shark Bed fo...|    Pet Products|          5|    

In [8]:
# 2. Helpful votes divided by total votes over half 

Helpful_df = over20_df.filter((over20_df.helpful_votes / over20_df.total_votes) >= 0.5)

Helpful_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51227803|R21KC552Y6HL8X|B00E2E1XBK|     180836080|Kennel Deck - 3 Pack|    Pet Products|          1|           27|         31|   N|                Y|The picture is of...|The product add p...| 2015-08-31|
|         US|   52747799| RX9WC9FTIR1XR|B00DZITYGU|     598776873|Grey Shark Bed fo...|    Pet Products|          5|    

In [11]:
# 3. Vine only df

vine_df = Helpful_df.filter(Helpful_df.vine == 'Y')

vine_df.show()



+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   44759540|R3A71VR1JZD8WF|B00JGWN8O8|     378897034|Petcube Interacti...|    Pet Products|          2|           27|         30|   Y|                N|Great idea, not s...|I do not hate thi...| 2015-08-29|
|         US|   37388532|R16OMUJIGI18JZ|B00IP05CUA|     833937853|Wellness Kittles ...|    Pet Products|          5|    

In [12]:
# 3. No vine reviews

vinefree_df = Helpful_df.filter(Helpful_df.vine == 'N')

vinefree_df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51227803|R21KC552Y6HL8X|B00E2E1XBK|     180836080|Kennel Deck - 3 Pack|    Pet Products|          1|           27|         31|   N|                Y|The picture is of...|The product add p...| 2015-08-31|
|         US|   52747799| RX9WC9FTIR1XR|B00DZITYGU|     598776873|Grey Shark Bed fo...|    Pet Products|          5|    

In [22]:
## 5 Number of reviews and breakdown 

paid_reviews = vine_df.count()

unpaid_reviews = vinefree_df.count()

total_reviews = Helpful_df.count()



In [23]:
## 5 continued 
print(paid_reviews, "paid reviews")
print(unpaid_reviews, "unpaid reviews")
print(total_reviews, "total reviews")

170 paid reviews
37840 unpaid reviews
38010 total reviews


In [27]:
## 5 cont. Filter by rating 

paid_five_star = vine_df.filter(vine_df.star_rating == 5).count()

paid_five_star_percent = (paid_five_star / paid_reviews) *100 

print(round(paid_five_star_percent,2), "% of paid are 5 star reviews ")

38.24 % of paid are 5 star reviews 


In [30]:
unpaid_five_star = vinefree_df.filter(vinefree_df.star_rating == 5).count()

unpaid_five_star_percent = (unpaid_five_star / unpaid_reviews) * 100 

print(round(unpaid_five_star_percent,2), "% of unpaid are 5 star reviews ")

54.47 % of unpaid are 5 star reviews 
